# Ejercicio propuesto 7. Deducción del rotacional en coordenadas esféricas

|Quién | Fecha | Qué hizo |
| ---  | ---   | ---      |
|Diego Andrés Alvarez, <daalvarez@unal.edu.co>  | Octubre 3, 2023      | Primer código |

In [1]:
from sympy import init_printing, latex, symbols, sin, cos, diff, simplify, Function, Matrix, expand, trigsimp
import sympy as sp

# Para imprimir bonito
init_printing()
from IPython.display import Math                 
def imprimir (texto1, variable, texto2=""):
    return Math(texto1 +  rf'{latex(variable)}' + texto2)

In [2]:
r, t, p = symbols('r, theta, phi')

Se definen las variables rectangulares $(x, y, z)$ en función de las variables esféricas $(r, \theta, \phi)$:

In [3]:
x = r*sin(t)*cos(p)
y = r*sin(t)*sin(p)
z = r*cos(t)

Se calcula la matriz jacobiana de la transformación $(r, \theta, \phi) \mapsto (x,y,z):$

In [4]:
xyz = Matrix([x, y, z])
rtp = Matrix([r, t, p])

J = xyz.jacobian(rtp)
imprimir(r"\boldsymbol{J}(r, \theta, \phi) = ", J)

<IPython.core.display.Math object>

Lo anterior es equivalente al comando:

In [5]:
J = Matrix([[ diff(x,r), diff(x,t), diff(x,p) ],
            [ diff(y,r), diff(y,t), diff(y,p) ],
            [ diff(z,r), diff(z,t), diff(z,p) ]])
imprimir(r"\boldsymbol{J}(r, \theta, \phi) = ", J)

<IPython.core.display.Math object>

De acuerdo al [teorema de la función inversa](https://es.wikipedia.org/wiki/Matriz_y_determinante_jacobianos), la matriz inversa de la matriz Jacobiana de una función invertible es la matriz Jacobiana de la función inversa. Recuerde adicionalmente que:
$$\left(\boldsymbol{A}^T\right)^{-1} = \left(\boldsymbol{A}^{-1}\right)^T = \boldsymbol{A}^{-T}$$

In [6]:
inv_JT = trigsimp(J.T.inv())
imprimir(r"\boldsymbol{J}^{-T}(r, \theta, \phi) = ", inv_JT)

<IPython.core.display.Math object>

Se extraen las derivadas de la ecuación anterior:

In [7]:
dr_dx, dt_dx, dp_dx = inv_JT[0,:]
dr_dy, dt_dy, dp_dy = inv_JT[1,:]
dr_dz, dt_dz, dp_dz = inv_JT[2,:]

Se definen las primeras derivadas utilizando la regla de la cadena:

In [8]:
d_dx = lambda f: diff(f,r)*dr_dx + diff(f,t)*dt_dx + diff(f,p)*dp_dx
d_dy = lambda f: diff(f,r)*dr_dy + diff(f,t)*dt_dy + diff(f,p)*dp_dy
d_dz = lambda f: diff(f,r)*dr_dz + diff(f,t)*dt_dz + diff(f,p)*dp_dz

Se especifica que $f_r$, $f_\theta$ y $f_\phi$ son funciones de $r$, $\theta$ y $\phi$

In [9]:
fr = Function('f_r')(r, t, p)
ft = Function('f_θ')(r, t, p)
fp = Function('f_ϕ')(r, t, p)

Se define la matriz de transformación de la base $\{\hat{\boldsymbol{r}},\hat{\boldsymbol{\theta}},\hat{\boldsymbol{\phi}}\}$ a la base $\{\hat{\boldsymbol{i}},\hat{\boldsymbol{j}},\hat{\boldsymbol{k}}\}$:


In [10]:
T = Matrix([[ sin(t)*cos(p),  cos(t)*cos(p),  -sin(p) ],     
            [ sin(t)*sin(p),  cos(t)*sin(p),   cos(p) ],      
            [        cos(t),        -sin(t),        0 ]])

Se define el campo vectorial en coordenadas esféricas con respecto a la base $\{\hat{\boldsymbol{r}},\hat{\boldsymbol{\theta}},\hat{\boldsymbol{\phi}}\}$:

In [11]:
f_esf = Matrix([ fr,     
                 ft,      
                 fp  ])

Se convierte el campo vectorial `f_esf` a su correspondiente en coordenadas rectangulares (con respecto a la base $\{\hat{\boldsymbol{i}},\hat{\boldsymbol{j}},\hat{\boldsymbol{k}}\}$)

In [12]:
f_rec = T * f_esf

Se extraen los componentes de `f_rec`:

In [13]:
fx, fy, fz = f_rec

Se especifica el rotacional en coordenadas rectangulares:

In [14]:
rot_f_ijk = trigsimp(Matrix([ d_dy(fz) - d_dz(fy),
                              d_dz(fx) - d_dx(fz),
                              d_dx(fy) - d_dy(fx) ]))

Se imprime el rotacional en coordenadas cilíndricas con respecto a la base $\{\hat{\boldsymbol{i}}, \hat{\boldsymbol{j}}, \hat{\boldsymbol{k}}\}$:

In [15]:
imprimir(r"\boldsymbol{\nabla} \times \boldsymbol{f}(r, \theta, \phi) = ", rot_f_ijk)

<IPython.core.display.Math object>

Se convierte el rotacional calculado anteriormente a uno con respecto a la base $\{\hat{\boldsymbol{r}}, \hat{\boldsymbol{\theta}}, \hat{\boldsymbol{\phi}}\}$:

In [16]:
rot_f_rtp = expand(trigsimp(T.T * rot_f_ijk))

imprimir(r"\boldsymbol{\nabla} \times \boldsymbol{f}(r, \theta, \phi) = ", rot_f_rtp)

<IPython.core.display.Math object>

Versión que figura en Wikipedia https://en.wikipedia.org/wiki/Spherical_coordinate_system:

In [17]:
rot_f_rtp2 = Matrix([[ (1/(r*sin(t)))*(diff(fp*sin(t), t) - diff(ft, p)) ],
                     [ (1/r)*((1/sin(t))*diff(fr, p) - diff(r*fp, r))    ],
                     [ (1/r)*(diff(r*ft, r) - diff(fr, t))               ]])

rot_f_rtp2 = expand(trigsimp(rot_f_rtp2))

imprimir(r"\boldsymbol{\nabla} \times \boldsymbol{f}(r, \theta, \phi) = ", rot_f_rtp2)

<IPython.core.display.Math object>

Se verifica que ambas formulaciones son equivalentes:

In [18]:
simplify(rot_f_rtp - rot_f_rtp2)